# Approaching (Almost) Any NLP Problem on Kaggle

In this post I'll talk about approaching natural language processing problems on Kaggle. As an example, we will use the data from this competition. We will create a very basic first model first and then improve it using different other features. We will also see how deep neural networks can be used and end this post with some ideas about ensembling in general.

### This covers:
- tfidf 
- count features
- logistic regression
- naive bayes
- svm
- xgboost
- grid search
- word vectors
- LSTM
- GRU
- Ensembling

*NOTE*: This notebook is not meant for achieving a very high score on the Leaderboard for this dataset. However, if you follow it properly, you can get a very high score with some tuning. ;)

So, without wasting any time, let's start with importing some important python modules that I'll be using.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

Using TensorFlow backend.


Let's load the datasets

In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
sample = pd.read_csv('input/sample_submission.csv')

A quick look at the data

In [3]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [5]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


The problem requires us to predict the author, i.e. EAP, HPL and MWS given the text. In simpler words, text classification with 3 different classes.

For this particular problem, Kaggle has specified multi-class log-loss as evaluation metric. This is implemented in the follow way (taken from: https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py)

In [6]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

We use the LabelEncoder from scikit-learn to convert text labels to integers, 0, 1 2

In [7]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

Before going further it is important that we split the data into training and validation sets. We can do it using `train_test_split` from the `model_selection` module of scikit-learn.

In [8]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [9]:
print (xtrain.shape)
print (xvalid.shape)

(17621,)
(1958,)


## Building Basic Models

Let's start building our very first model. 

Our very first model is a simple TF-IDF (Term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression.

In [10]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [11]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.626 


And there we go. We have our first model with a multiclass logloss of 0.626.

But we are greedy and want a better score. Lets look at the same model with a different data.

Instead of using TF-IDF, we can also use word counts as features. This can be done easily using CountVectorizer from scikit-learn.

In [12]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [13]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.528 


Aaaaanddddddd Wallah! We just improved our first model by 0.1!!!

Next, let's try a very simple model which was quite famous in ancient times - Naive Bayes.

Let's see what happens when we use naive bayes on these two datasets:

In [14]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.578 


Good performance! But the logistic regression on counts is still better! What happens when we use this model on counts data instead?

In [15]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.485 


Whoa! Seems like old stuff still works good!!!! One more ancient algorithms in the list is SVMs. Some people "love" SVMs. So, we must try SVM on this dataset.

Since SVMs take a lot of time, we will reduce the number of features from the TF-IDF using Singular Value Decomposition before applying SVM. 

Also, note that before applying SVMs, we *must* standardize the data.

In [16]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

Now it's time to apply SVM. After running the following cell, feel free to go for a walk or talk to your girlfriend/boyfriend. :P

In [17]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.734 


Oops! time to get up! Looks like SVM doesn't perform well on this data...! 

Before moving further, lets apply the most popular algorithm on Kaggle: xgboost!

In [18]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.782 


In [19]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.772 


In [20]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.776 


In [21]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.816 


Seems like no luck with XGBoost! But that is not correct. I haven't done any hyperparameter optimizations yet. And since I'm lazy, I'll just tell you how to do it and you can do it on your own! ;). This will be discussed in the next section:


## Grid Search

Its a technique for hyperparameter optimization. Not so effective but can give good results if you know the grid you want to use. I specify the parameters that should usually be used in this post: http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/ Please keep in mind that these are the parameters I usually use. There are many other methods of hyperparameter optimization which may or may not be as effective.

In this section, I'll talk about grid search using logistic regression. 

Before starting with grid search we need to create a scoring function. This is accomplished using the `make_scorer` function of scikit-learn.


In [22]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

Next we need a pipeline. For demonstration here, i'll be using a pipeline consisting of SVD, scaling and then logistic regression. Its better to understand with more modules in pipeline than just one ;)

In [23]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

Next we need a grid of parameters:

In [24]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

So, for SVD we evaluate 120 and 180 components and for logistic regression we evaluate three different values of C with l1 and l2 penalty. We can now start grid search on these parameters.

In [25]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10,iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=-0.7831575634886204, total=   1.4s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=-0.7780969798093583, total=   1.6s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.2s remaining:    0.0s


[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=180, score=-0.7478385769768283, total=   2.4s


...

[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=120 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=120, score=-0.7802429164634146, total=   1.7s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   18.8s remaining:    0.0s


[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=120 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=120, score=-0.7766826806213276, total=   1.7s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=180, score=-0.7415316724224758, total=   3.4s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=180, score=-0.7386597909120567, total=   2.9s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.7801107551313183, total=   1.5s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.7726113123515684, total=   1.5s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=180, score=-0.7491065676324745, total=

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   57.0s finished


Best score: -0.738
Best parameters set:
	lr__C: 10
	lr__penalty: 'l2'
	svd__n_components: 180


The score comes similar to what we had for SVM. This technique can be used to finetune xgboost or even multinomial naive bayes as below. We will use the tfidf data here:

In [26]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] nb__alpha=0.001 .................................................
[CV] ....... nb__alpha=0.001, score=-0.6204702511359115, total=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] nb__alpha=0.001 .................................................
[CV] ....... nb__alpha=0.001, score=-0.6414540936460588, total=   0.0s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] nb__alpha=0.01 ..................................................
[CV] ........ nb__alpha=0.01, score=-0.5107778556718855, total=   0.0s


...

[CV] nb__alpha=10 ....................................................
[CV] .......... nb__alpha=10, score=-0.9496648029421535, total=   0.0s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s


[CV] nb__alpha=10 ....................................................
[CV] ........... nb__alpha=10, score=-0.950588400704189, total=   0.0s
[CV] nb__alpha=100 ...................................................
[CV] .......... nb__alpha=100, score=-1.067264698247982, total=   0.0s
[CV] nb__alpha=100 ...................................................
[CV] ......... nb__alpha=100, score=-1.0673577682186433, total=   0.0s


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.2s finished


Best score: -0.492
Best parameters set:
	nb__alpha: 0.1


This is an improvement of 8% over the original naive bayes score!

In NLP problems, it's customary to look at word vectors. Word vectors give a lot of insights about the data. Let's dive into that.

## Word Vectors

Without going into too much details, I would explain how to create sentence vectors and how can we use them to create a machine learning model on top of it. I am a fan of GloVe vectors, word2vec and fasttext. In this post, I'll be using the GloVe vectors. You can download the GloVe vectors from here `http://www-nlp.stanford.edu/data/glove.840B.300d.zip`

In [39]:
# load the GloVe vectors in a dictionary:

# embeddings_index = {}
# f = open('glove.840B.300d.txt')
# for line in tqdm(f):
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))
import re
embeddings_index = {}
f = open('glove.840B.300d.txt',encoding = 'utf-8')
for line in tqdm(f):
    values = re.sub(r'[^\x00-\x7F]+','', line).split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))







0it [00:00, ?it/s]





509it [00:00, 4941.47it/s]





947it [00:00, 4711.17it/s]





1414it [00:00, 4697.40it/s]





1933it [00:00, 4808.18it/s]





2562it [00:00, 5113.48it/s]





3173it [00:00, 5270.46it/s]





3785it [00:00, 5399.12it/s]





4337it [00:00, 5414.17it/s]





4898it [00:00, 5435.87it/s]





5443it [00:01, 5421.00it/s]





6041it [00:01, 5476.57it/s]





6641it [00:01, 5520.05it/s]





7237it [00:01, 5553.79it/s]





7858it [00:01, 5600.54it/s]





8459it [00:01, 5627.76it/s]





9054it [00:01, 5644.32it/s]





9664it [00:01, 5674.37it/s]





10314it [00:01, 5720.14it/s]





10933it [00:01, 5744.81it/s]





11549it [00:02, 5748.30it/s]





12155it [00:02, 5725.06it/s]





12799it [00:02, 5757.21it/s]





13447it [00:02, 5788.30it/s]





14067it [00:02, 5805.28it/s]





14684it [00:02, 5817.42it/s]





15301it [00:02, 5830.84it/s]





15924it [00:02, 5845.48it/s]





16577it [00:02, 5869.71it/s]





17206it [00:02, 5856.03it/s]





178

138791it [00:25, 5510.54it/s]





139399it [00:25, 5512.80it/s]





139983it [00:25, 5511.91it/s]





140606it [00:25, 5514.94it/s]





141252it [00:25, 5518.63it/s]





141908it [00:25, 5522.69it/s]





142528it [00:25, 5524.03it/s]





143155it [00:25, 5527.12it/s]





143802it [00:26, 5530.53it/s]





144446it [00:26, 5534.22it/s]





145077it [00:26, 5535.92it/s]





145704it [00:26, 5538.71it/s]





146327it [00:26, 5540.90it/s]





146946it [00:26, 5542.93it/s]





147589it [00:26, 5546.47it/s]





148214it [00:26, 5548.48it/s]





148870it [00:26, 5552.46it/s]





149513it [00:26, 5555.72it/s]





150148it [00:27, 5558.66it/s]





150837it [00:27, 5563.57it/s]





151489it [00:27, 5566.06it/s]





152138it [00:27, 5569.65it/s]





152784it [00:27, 5572.08it/s]





153454it [00:27, 5576.58it/s]





154152it [00:27, 5581.26it/s]





154816it [00:27, 5585.08it/s]





155478it [00:27, 5587.79it/s]





156133it [00:27, 5591.24it/s]





156787it [00:28, 559

291063it [00:49, 5888.77it/s]





291730it [00:49, 5890.22it/s]





292415it [00:49, 5892.28it/s]





293100it [00:49, 5894.08it/s]





293788it [00:49, 5896.06it/s]





294467it [00:49, 5896.90it/s]





295147it [00:50, 5898.71it/s]





295839it [00:50, 5900.75it/s]





296536it [00:50, 5902.87it/s]





297242it [00:50, 5905.17it/s]





297940it [00:50, 5907.30it/s]





298632it [00:50, 5909.07it/s]





299320it [00:50, 5910.87it/s]





300015it [00:50, 5912.92it/s]





300720it [00:50, 5915.15it/s]





301423it [00:50, 5917.23it/s]





302120it [00:51, 5919.40it/s]





302817it [00:51, 5920.65it/s]





303508it [00:51, 5922.58it/s]





304199it [00:51, 5924.50it/s]





304909it [00:51, 5926.78it/s]





305609it [00:51, 5928.87it/s]





306323it [00:51, 5931.21it/s]





307025it [00:51, 5933.08it/s]





307723it [00:51, 5935.10it/s]





308421it [00:51, 5937.00it/s]





309122it [00:52, 5938.95it/s]





309834it [00:52, 5941.32it/s]





310545it [00:52, 594

437556it [01:13, 5921.22it/s]





438077it [01:13, 5920.26it/s]





438615it [01:14, 5919.45it/s]





439260it [01:14, 5920.33it/s]





439888it [01:14, 5920.81it/s]





440489it [01:14, 5920.93it/s]





441122it [01:14, 5921.48it/s]





441756it [01:14, 5921.96it/s]





442442it [01:14, 5923.29it/s]





443146it [01:14, 5924.79it/s]





443799it [01:14, 5925.20it/s]





444446it [01:14, 5926.00it/s]





445092it [01:15, 5926.40it/s]





445764it [01:15, 5927.53it/s]





446437it [01:15, 5928.60it/s]





447094it [01:15, 5929.45it/s]





447751it [01:15, 5930.06it/s]





448429it [01:15, 5931.18it/s]





449123it [01:15, 5932.51it/s]





449803it [01:15, 5933.66it/s]





450476it [01:15, 5934.63it/s]





451147it [01:16, 5935.18it/s]





451807it [01:16, 5934.73it/s]





452457it [01:16, 5935.47it/s]





453099it [01:16, 5936.10it/s]





453763it [01:16, 5936.79it/s]





454409it [01:16, 5937.40it/s]





455081it [01:16, 5938.49it/s]





455731it [01:16, 593

592950it [01:38, 6046.65it/s]





593624it [01:38, 6047.29it/s]





594325it [01:38, 6048.33it/s]





595019it [01:38, 6049.30it/s]





595734it [01:38, 6050.36it/s]





596428it [01:38, 6051.20it/s]





597137it [01:38, 6052.19it/s]





597848it [01:38, 6053.32it/s]





598549it [01:38, 6054.35it/s]





599250it [01:38, 6055.25it/s]





599951it [01:39, 6056.22it/s]





600652it [01:39, 6057.12it/s]





601356it [01:39, 6058.17it/s]





602061it [01:39, 6059.17it/s]





602767it [01:39, 6060.17it/s]





603474it [01:39, 6061.19it/s]





604182it [01:39, 6062.21it/s]





604888it [01:39, 6063.15it/s]





605597it [01:39, 6064.17it/s]





606311it [01:39, 6065.25it/s]





607027it [01:40, 6066.34it/s]





607738it [01:40, 6067.39it/s]





608448it [01:40, 6068.41it/s]





609171it [01:40, 6069.51it/s]





609885it [01:40, 6070.58it/s]





610597it [01:40, 6071.62it/s]





611309it [01:40, 6072.66it/s]





612021it [01:40, 6073.70it/s]





612738it [01:40, 607

...

1695328it [04:23, 6441.26it/s]





1696050it [04:23, 6441.56it/s]





1696770it [04:23, 6441.85it/s]





1697490it [04:23, 6442.09it/s]





1698204it [04:23, 6442.30it/s]





1698914it [04:23, 6442.53it/s]





1699629it [04:23, 6442.79it/s]





1700348it [04:23, 6443.08it/s]





1701073it [04:24, 6443.38it/s]





1701796it [04:24, 6443.68it/s]





1702521it [04:24, 6443.99it/s]





1703244it [04:24, 6444.28it/s]





1703966it [04:24, 6444.58it/s]





1704688it [04:24, 6444.80it/s]





1705413it [04:24, 6445.10it/s]





1706134it [04:24, 6445.36it/s]





1706853it [04:24, 6445.65it/s]





1707572it [04:24, 6445.93it/s]





1708292it [04:25, 6446.21it/s]





1709012it [04:25, 6446.49it/s]





1709732it [04:25, 6446.78it/s]





1710457it [04:25, 6447.06it/s]





1711177it [04:25, 6447.37it/s]





1711897it [04:25, 6447.58it/s]





1712623it [04:25, 6447.88it/s]





1713341it [04:25, 6448.13it/s]





1714061it [04:25, 6448.44it/s]





1714787it [04:25, 6448.72it/

1852846it [04:47, 6451.79it/s]





1853555it [04:47, 6451.99it/s]





1854262it [04:47, 6452.20it/s]





1854982it [04:47, 6452.46it/s]





1855702it [04:47, 6452.72it/s]





1856415it [04:47, 6452.82it/s]





1857118it [04:47, 6453.00it/s]





1857834it [04:47, 6453.27it/s]





1858548it [04:47, 6453.48it/s]





1859264it [04:48, 6453.75it/s]





1859979it [04:48, 6453.97it/s]





1860694it [04:48, 6454.23it/s]





1861414it [04:48, 6454.47it/s]





1862129it [04:48, 6454.73it/s]





1862844it [04:48, 6454.95it/s]





1863559it [04:48, 6455.21it/s]





1864274it [04:48, 6455.39it/s]





1864989it [04:48, 6455.63it/s]





1865702it [04:48, 6455.86it/s]





1866419it [04:49, 6456.11it/s]





1867132it [04:49, 6456.34it/s]





1867845it [04:49, 6456.57it/s]





1868558it [04:49, 6456.78it/s]





1869271it [04:49, 6457.02it/s]





1869984it [04:49, 6457.25it/s]





1870697it [04:49, 6457.48it/s]





1871410it [04:49, 6457.67it/s]





1872119it [04:49, 6457.91it/

2015216it [05:10, 6492.93it/s]





2015931it [05:10, 6493.14it/s]





2016646it [05:10, 6493.35it/s]





2017361it [05:10, 6493.57it/s]





2018076it [05:10, 6493.78it/s]





2018791it [05:10, 6493.99it/s]





2019507it [05:10, 6494.20it/s]





2020223it [05:11, 6494.37it/s]





2020937it [05:11, 6494.58it/s]





2021666it [05:11, 6494.84it/s]





2022384it [05:11, 6495.06it/s]





2023101it [05:11, 6495.25it/s]





2023818it [05:11, 6495.45it/s]





2024531it [05:11, 6495.63it/s]





2025242it [05:11, 6495.79it/s]





2025959it [05:11, 6496.02it/s]





2026686it [05:11, 6496.25it/s]





2027402it [05:12, 6496.42it/s]





2028117it [05:12, 6496.63it/s]





2028834it [05:12, 6496.85it/s]





2029547it [05:12, 6497.03it/s]





2030266it [05:12, 6497.25it/s]





2030988it [05:12, 6497.48it/s]





2031712it [05:12, 6497.72it/s]





2032431it [05:12, 6497.92it/s]





2033148it [05:12, 6498.13it/s]





2033872it [05:12, 6498.37it/s]





2034599it [05:13, 6498.62it/

2178323it [05:33, 6536.21it/s]





2179044it [05:33, 6536.43it/s]





2179764it [05:33, 6536.59it/s]





2180493it [05:33, 6536.82it/s]





2181212it [05:33, 6536.88it/s]





2181936it [05:33, 6537.09it/s]





2182663it [05:33, 6537.31it/s]





2183382it [05:33, 6537.50it/s]





2184099it [05:34, 6537.69it/s]





2184816it [05:34, 6537.86it/s]





2185532it [05:34, 6538.05it/s]





2186248it [05:34, 6537.96it/s]





2186949it [05:34, 6538.10it/s]





2187661it [05:34, 6538.27it/s]





2188360it [05:34, 6538.11it/s]





2189032it [05:34, 6537.37it/s]





2189645it [05:34, 6536.95it/s]





2190236it [05:35, 6532.34it/s]





2190939it [05:35, 6532.51it/s]





2191634it [05:35, 6532.63it/s]





2192344it [05:35, 6532.80it/s]





2193048it [05:35, 6532.95it/s]





2193757it [05:35, 6533.12it/s]





2194431it [05:35, 6533.18it/s]





2195144it [05:35, 6533.36it/s]





2195846it [05:36, 6533.50it/s]





2196017it [05:36, 6533.49it/s]

Found 2185161 word vectors.


In [42]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [43]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]








  0%|                                                | 0/17621 [00:00<?, ?it/s]






  0%|                                        | 1/17621 [00:00<45:31,  6.45it/s]






  1%|▌                                    | 243/17621 [00:00<00:18, 952.89it/s]






  3%|█                                   | 497/17621 [00:00<00:12, 1399.92it/s]






  4%|█▌                                  | 750/17621 [00:00<00:10, 1648.26it/s]






  6%|██                                 | 1031/17621 [00:00<00:08, 1857.55it/s]






  7%|██▌                                | 1305/17621 [00:00<00:08, 1989.22it/s]






  9%|███▏                               | 1585/17621 [00:00<00:07, 2099.22it/s]






 11%|███▊                               | 1889/17621 [00:00<00:07, 2209.23it/s]






 13%|████▍                              | 2213/17621 [00:00<00:06, 2317.15it/s]






 14%|████▉                              | 2492/17621 [00:01<00:06, 2326.66it/s]






 16%|█████▌                             | 2771/

In [44]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

Let's see the performance of xgboost on glove features:

In [45]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.812 


In [46]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.698 


we see that a simple tuning of parameters can improve xgboost score on GloVe features! Believe me you can squeeze a lot more from it.

## Deep Learning

But this is an era of deep learning! We cant live without training a few neural networks. Here, we will train LSTM and a simple dense network on the GloVe features. Let's start with the dense network first:

In [47]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [48]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [49]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [50]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 17621 samples, validate on 1958 samples
Epoch 1/5
17621/17621 [==============================] - ETA: 5:02 - loss: 1.534 - ETA: 1:19 - loss: 1.573 - ETA: 36s - loss: 1.348 - ETA: 23s - loss: 1.33 - ETA: 17s - loss: 1.30 - ETA: 13s - loss: 1.24 - ETA: 11s - loss: 1.20 - ETA: 9s - loss: 1.1921 - ETA: 8s - loss: 1.165 - ETA: 7s - loss: 1.142 - ETA: 6s - loss: 1.130 - ETA: 6s - loss: 1.119 - ETA: 5s - loss: 1.109 - ETA: 5s - loss: 1.093 - ETA: 4s - loss: 1.080 - ETA: 4s - loss: 1.074 - ETA: 4s - loss: 1.058 - ETA: 4s - loss: 1.056 - ETA: 3s - loss: 1.055 - ETA: 3s - loss: 1.053 - ETA: 3s - loss: 1.046 - ETA: 3s - loss: 1.038 - ETA: 3s - loss: 1.030 - ETA: 2s - loss: 1.026 - ETA: 2s - loss: 1.021 - ETA: 2s - loss: 1.015 - ETA: 2s - loss: 1.007 - ETA: 2s - loss: 1.002 - ETA: 2s - loss: 0.999 - ETA: 1s - loss: 0.991 - ETA: 1s - loss: 0.986 - ETA: 1s - loss: 0.980 - ETA: 1s - loss: 0.978 - ETA: 1s - loss: 0.973 - ETA: 1s - loss: 0.969 - ETA: 1s - loss: 0.964 - ETA: 1s - loss: 0.963 - 

You need to keep on tuning the parameters of the neural network, add more layers, increase dropout to get better results. Here, I'm just showing that its fast to implement and run and gets better result than xgboost without any optimization :)

To move further, i.e. with LSTMs we need to tokenize the text data

In [51]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [52]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector








  0%|                                                | 0/25943 [00:00<?, ?it/s]






100%|████████████████████████████████| 25943/25943 [00:00<00:00, 278940.74it/s]

In [53]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [54]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - ETA: 1:22 - loss: 1.121 - ETA: 1:02 - loss: 1.121 - ETA: 54s - loss: 1.122 - ETA: 50s - loss: 1.12 - ETA: 46s - loss: 1.11 - ETA: 44s - loss: 1.11 - ETA: 41s - loss: 1.11 - ETA: 39s - loss: 1.11 - ETA: 37s - loss: 1.11 - ETA: 36s - loss: 1.10 - ETA: 34s - loss: 1.10 - ETA: 33s - loss: 1.10 - ETA: 31s - loss: 1.10 - ETA: 30s - loss: 1.10 - ETA: 28s - loss: 1.10 - ETA: 27s - loss: 1.10 - ETA: 25s - loss: 1.10 - ETA: 24s - loss: 1.09 - ETA: 22s - loss: 1.09 - ETA: 21s - loss: 1.09 - ETA: 19s - loss: 1.09 - ETA: 18s - loss: 1.09 - ETA: 16s - loss: 1.09 - ETA: 15s - loss: 1.09 - ETA: 13s - loss: 1.09 - ETA: 12s - loss: 1.09 - ETA: 10s - loss: 1.09 - ETA: 9s - loss: 1.0913 - ETA: 7s - loss: 1.090 - ETA: 6s - loss: 1.089 - ETA: 4s - loss: 1.088 - ETA: 3s - loss: 1.087 - ETA: 2s - loss: 1.087 - ETA: 0s - loss: 1.086 - 52s 3ms/step - loss: 1.0860 - val_loss: 1.0231
Epoch 2/100
17621/17621

17621/17621 [==============================] - ETA: 51s - loss: 0.80 - ETA: 49s - loss: 0.77 - ETA: 48s - loss: 0.76 - ETA: 47s - loss: 0.76 - ETA: 45s - loss: 0.74 - ETA: 43s - loss: 0.74 - ETA: 42s - loss: 0.75 - ETA: 40s - loss: 0.74 - ETA: 39s - loss: 0.74 - ETA: 37s - loss: 0.74 - ETA: 36s - loss: 0.74 - ETA: 35s - loss: 0.74 - ETA: 33s - loss: 0.74 - ETA: 32s - loss: 0.74 - ETA: 31s - loss: 0.74 - ETA: 29s - loss: 0.74 - ETA: 28s - loss: 0.74 - ETA: 26s - loss: 0.75 - ETA: 24s - loss: 0.74 - ETA: 23s - loss: 0.74 - ETA: 21s - loss: 0.74 - ETA: 20s - loss: 0.75 - ETA: 18s - loss: 0.75 - ETA: 16s - loss: 0.74 - ETA: 15s - loss: 0.74 - ETA: 13s - loss: 0.74 - ETA: 12s - loss: 0.74 - ETA: 10s - loss: 0.74 - ETA: 8s - loss: 0.7445 - ETA: 7s - loss: 0.743 - ETA: 5s - loss: 0.743 - ETA: 3s - loss: 0.746 - ETA: 2s - loss: 0.746 - ETA: 0s - loss: 0.745 - 59s 3ms/step - loss: 0.7457 - val_loss: 0.6740
Epoch 10/100
17621/17621 [==============================] - ETA: 51s - loss: 0.74 - ETA: 

17621/17621 [==============================] - ETA: 1:00 - loss: 0.672 - ETA: 56s - loss: 0.660 - ETA: 54s - loss: 0.65 - ETA: 54s - loss: 0.65 - ETA: 52s - loss: 0.64 - ETA: 50s - loss: 0.65 - ETA: 48s - loss: 0.65 - ETA: 45s - loss: 0.64 - ETA: 43s - loss: 0.64 - ETA: 41s - loss: 0.64 - ETA: 41s - loss: 0.65 - ETA: 39s - loss: 0.64 - ETA: 38s - loss: 0.64 - ETA: 36s - loss: 0.64 - ETA: 34s - loss: 0.64 - ETA: 32s - loss: 0.63 - ETA: 31s - loss: 0.63 - ETA: 29s - loss: 0.63 - ETA: 27s - loss: 0.63 - ETA: 25s - loss: 0.63 - ETA: 23s - loss: 0.63 - ETA: 21s - loss: 0.64 - ETA: 19s - loss: 0.63 - ETA: 18s - loss: 0.63 - ETA: 16s - loss: 0.63 - ETA: 14s - loss: 0.63 - ETA: 12s - loss: 0.63 - ETA: 11s - loss: 0.63 - ETA: 9s - loss: 0.6366 - ETA: 7s - loss: 0.635 - ETA: 5s - loss: 0.636 - ETA: 4s - loss: 0.636 - ETA: 2s - loss: 0.636 - ETA: 0s - loss: 0.635 - 61s 3ms/step - loss: 0.6364 - val_loss: 0.6044
Epoch 18/100
17621/17621 [==============================] - ETA: 54s - loss: 0.60 - ET

17621/17621 [==============================] - ETA: 52s - loss: 0.55 - ETA: 51s - loss: 0.54 - ETA: 49s - loss: 0.53 - ETA: 48s - loss: 0.53 - ETA: 46s - loss: 0.53 - ETA: 45s - loss: 0.53 - ETA: 43s - loss: 0.53 - ETA: 41s - loss: 0.53 - ETA: 40s - loss: 0.53 - ETA: 38s - loss: 0.53 - ETA: 37s - loss: 0.53 - ETA: 35s - loss: 0.53 - ETA: 34s - loss: 0.54 - ETA: 32s - loss: 0.54 - ETA: 30s - loss: 0.54 - ETA: 29s - loss: 0.54 - ETA: 27s - loss: 0.54 - ETA: 26s - loss: 0.54 - ETA: 24s - loss: 0.54 - ETA: 22s - loss: 0.54 - ETA: 21s - loss: 0.54 - ETA: 19s - loss: 0.54 - ETA: 18s - loss: 0.54 - ETA: 16s - loss: 0.54 - ETA: 14s - loss: 0.54 - ETA: 13s - loss: 0.54 - ETA: 11s - loss: 0.54 - ETA: 10s - loss: 0.55 - ETA: 8s - loss: 0.5521 - ETA: 7s - loss: 0.552 - ETA: 5s - loss: 0.554 - ETA: 3s - loss: 0.552 - ETA: 2s - loss: 0.552 - ETA: 0s - loss: 0.552 - 57s 3ms/step - loss: 0.5530 - val_loss: 0.5463
Epoch 26/100
17621/17621 [==============================] - ETA: 52s - loss: 0.55 - ETA: 

17621/17621 [==============================] - ETA: 52s - loss: 0.45 - ETA: 51s - loss: 0.46 - ETA: 49s - loss: 0.45 - ETA: 48s - loss: 0.46 - ETA: 46s - loss: 0.46 - ETA: 45s - loss: 0.46 - ETA: 43s - loss: 0.46 - ETA: 41s - loss: 0.47 - ETA: 40s - loss: 0.47 - ETA: 38s - loss: 0.47 - ETA: 37s - loss: 0.47 - ETA: 35s - loss: 0.47 - ETA: 33s - loss: 0.47 - ETA: 32s - loss: 0.47 - ETA: 30s - loss: 0.47 - ETA: 29s - loss: 0.47 - ETA: 27s - loss: 0.47 - ETA: 26s - loss: 0.47 - ETA: 24s - loss: 0.47 - ETA: 22s - loss: 0.48 - ETA: 21s - loss: 0.48 - ETA: 19s - loss: 0.48 - ETA: 18s - loss: 0.48 - ETA: 16s - loss: 0.48 - ETA: 14s - loss: 0.47 - ETA: 13s - loss: 0.47 - ETA: 11s - loss: 0.48 - ETA: 10s - loss: 0.48 - ETA: 8s - loss: 0.4832 - ETA: 7s - loss: 0.482 - ETA: 5s - loss: 0.481 - ETA: 3s - loss: 0.481 - ETA: 2s - loss: 0.479 - ETA: 0s - loss: 0.478 - 57s 3ms/step - loss: 0.4781 - val_loss: 0.5287
Epoch 34/100
17621/17621 [==============================] - ETA: 54s - loss: 0.44 - ETA: 

...

17621/17621 [==============================] - ETA: 51s - loss: 0.32 - ETA: 50s - loss: 0.34 - ETA: 49s - loss: 0.33 - ETA: 47s - loss: 0.31 - ETA: 46s - loss: 0.31 - ETA: 44s - loss: 0.32 - ETA: 43s - loss: 0.32 - ETA: 41s - loss: 0.32 - ETA: 39s - loss: 0.32 - ETA: 38s - loss: 0.31 - ETA: 36s - loss: 0.32 - ETA: 35s - loss: 0.32 - ETA: 33s - loss: 0.31 - ETA: 32s - loss: 0.32 - ETA: 30s - loss: 0.32 - ETA: 28s - loss: 0.32 - ETA: 27s - loss: 0.32 - ETA: 25s - loss: 0.32 - ETA: 24s - loss: 0.32 - ETA: 22s - loss: 0.32 - ETA: 21s - loss: 0.32 - ETA: 19s - loss: 0.32 - ETA: 17s - loss: 0.32 - ETA: 16s - loss: 0.32 - ETA: 14s - loss: 0.32 - ETA: 13s - loss: 0.31 - ETA: 11s - loss: 0.31 - ETA: 10s - loss: 0.31 - ETA: 8s - loss: 0.3186 - ETA: 6s - loss: 0.318 - ETA: 5s - loss: 0.316 - ETA: 3s - loss: 0.318 - ETA: 2s - loss: 0.319 - ETA: 0s - loss: 0.319 - 56s 3ms/step - loss: 0.3187 - val_loss: 0.5427
Epoch 66/100
17621/17621 [==============================] - ETA: 52s - loss: 0.29 - ETA: 

17621/17621 [==============================] - ETA: 52s - loss: 0.25 - ETA: 50s - loss: 0.26 - ETA: 49s - loss: 0.26 - ETA: 47s - loss: 0.28 - ETA: 46s - loss: 0.28 - ETA: 44s - loss: 0.28 - ETA: 43s - loss: 0.29 - ETA: 41s - loss: 0.29 - ETA: 39s - loss: 0.30 - ETA: 38s - loss: 0.29 - ETA: 36s - loss: 0.30 - ETA: 35s - loss: 0.29 - ETA: 33s - loss: 0.29 - ETA: 32s - loss: 0.29 - ETA: 30s - loss: 0.29 - ETA: 28s - loss: 0.29 - ETA: 27s - loss: 0.29 - ETA: 25s - loss: 0.29 - ETA: 24s - loss: 0.29 - ETA: 22s - loss: 0.29 - ETA: 21s - loss: 0.29 - ETA: 19s - loss: 0.29 - ETA: 17s - loss: 0.29 - ETA: 16s - loss: 0.29 - ETA: 14s - loss: 0.29 - ETA: 13s - loss: 0.29 - ETA: 11s - loss: 0.29 - ETA: 10s - loss: 0.29 - ETA: 8s - loss: 0.2918 - ETA: 6s - loss: 0.291 - ETA: 5s - loss: 0.292 - ETA: 3s - loss: 0.293 - ETA: 2s - loss: 0.294 - ETA: 0s - loss: 0.294 - 56s 3ms/step - loss: 0.2944 - val_loss: 0.5555
Epoch 74/100
17621/17621 [==============================] - ETA: 52s - loss: 0.32 - ETA: 

17621/17621 [==============================] - ETA: 53s - loss: 0.26 - ETA: 51s - loss: 0.26 - ETA: 49s - loss: 0.26 - ETA: 47s - loss: 0.26 - ETA: 46s - loss: 0.26 - ETA: 44s - loss: 0.25 - ETA: 43s - loss: 0.26 - ETA: 41s - loss: 0.26 - ETA: 39s - loss: 0.26 - ETA: 38s - loss: 0.26 - ETA: 36s - loss: 0.26 - ETA: 35s - loss: 0.26 - ETA: 33s - loss: 0.26 - ETA: 32s - loss: 0.26 - ETA: 30s - loss: 0.26 - ETA: 28s - loss: 0.26 - ETA: 27s - loss: 0.26 - ETA: 25s - loss: 0.26 - ETA: 24s - loss: 0.26 - ETA: 22s - loss: 0.26 - ETA: 21s - loss: 0.26 - ETA: 19s - loss: 0.26 - ETA: 17s - loss: 0.26 - ETA: 16s - loss: 0.26 - ETA: 14s - loss: 0.26 - ETA: 13s - loss: 0.27 - ETA: 11s - loss: 0.27 - ETA: 10s - loss: 0.27 - ETA: 8s - loss: 0.2733 - ETA: 6s - loss: 0.275 - ETA: 5s - loss: 0.276 - ETA: 3s - loss: 0.278 - ETA: 2s - loss: 0.279 - ETA: 0s - loss: 0.280 - 56s 3ms/step - loss: 0.2806 - val_loss: 0.5434
Epoch 82/100
17621/17621 [==============================] - ETA: 52s - loss: 0.24 - ETA: 

17621/17621 [==============================] - ETA: 52s - loss: 0.22 - ETA: 52s - loss: 0.22 - ETA: 50s - loss: 0.23 - ETA: 48s - loss: 0.23 - ETA: 47s - loss: 0.25 - ETA: 45s - loss: 0.25 - ETA: 43s - loss: 0.26 - ETA: 42s - loss: 0.26 - ETA: 40s - loss: 0.25 - ETA: 39s - loss: 0.25 - ETA: 37s - loss: 0.25 - ETA: 35s - loss: 0.25 - ETA: 34s - loss: 0.25 - ETA: 32s - loss: 0.25 - ETA: 30s - loss: 0.25 - ETA: 29s - loss: 0.26 - ETA: 27s - loss: 0.26 - ETA: 26s - loss: 0.25 - ETA: 24s - loss: 0.25 - ETA: 22s - loss: 0.25 - ETA: 21s - loss: 0.25 - ETA: 19s - loss: 0.26 - ETA: 18s - loss: 0.26 - ETA: 16s - loss: 0.26 - ETA: 15s - loss: 0.26 - ETA: 13s - loss: 0.26 - ETA: 11s - loss: 0.26 - ETA: 10s - loss: 0.26 - ETA: 8s - loss: 0.2621 - ETA: 7s - loss: 0.261 - ETA: 5s - loss: 0.260 - ETA: 3s - loss: 0.261 - ETA: 2s - loss: 0.262 - ETA: 0s - loss: 0.263 - 57s 3ms/step - loss: 0.2636 - val_loss: 0.5576
Epoch 90/100
17621/17621 [==============================] - ETA: 51s - loss: 0.26 - ETA: 

17621/17621 [==============================] - ETA: 53s - loss: 0.24 - ETA: 51s - loss: 0.23 - ETA: 50s - loss: 0.23 - ETA: 49s - loss: 0.23 - ETA: 48s - loss: 0.23 - ETA: 46s - loss: 0.23 - ETA: 44s - loss: 0.24 - ETA: 42s - loss: 0.24 - ETA: 41s - loss: 0.23 - ETA: 39s - loss: 0.24 - ETA: 37s - loss: 0.24 - ETA: 36s - loss: 0.25 - ETA: 34s - loss: 0.24 - ETA: 33s - loss: 0.24 - ETA: 32s - loss: 0.24 - ETA: 30s - loss: 0.24 - ETA: 28s - loss: 0.24 - ETA: 27s - loss: 0.24 - ETA: 25s - loss: 0.24 - ETA: 23s - loss: 0.24 - ETA: 22s - loss: 0.24 - ETA: 20s - loss: 0.24 - ETA: 18s - loss: 0.24 - ETA: 17s - loss: 0.25 - ETA: 15s - loss: 0.24 - ETA: 13s - loss: 0.25 - ETA: 12s - loss: 0.24 - ETA: 10s - loss: 0.24 - ETA: 8s - loss: 0.2510 - ETA: 7s - loss: 0.250 - ETA: 5s - loss: 0.252 - ETA: 3s - loss: 0.251 - ETA: 2s - loss: 0.251 - ETA: 0s - loss: 0.251 - 58s 3ms/step - loss: 0.2511 - val_loss: 0.5853
Epoch 98/100
17621/17621 [==============================] - ETA: 51s - loss: 0.23 - ETA: 

We see that the score is now less than 0.5. I ran it for many epochs without stopping at the best but you can use early stopping to stop at the best iteration. How do I use early stopping?

well, pretty easy. let's compile the model again:

In [55]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - ETA: 3:19 - loss: 1.133 - ETA: 2:53 - loss: 1.125 - ETA: 2:40 - loss: 1.120 - ETA: 2:31 - loss: 1.123 - ETA: 2:24 - loss: 1.122 - ETA: 2:18 - loss: 1.122 - ETA: 2:12 - loss: 1.120 - ETA: 2:06 - loss: 1.117 - ETA: 2:00 - loss: 1.116 - ETA: 1:54 - loss: 1.114 - ETA: 1:49 - loss: 1.112 - ETA: 1:44 - loss: 1.110 - ETA: 1:39 - loss: 1.106 - ETA: 1:34 - loss: 1.104 - ETA: 1:29 - loss: 1.103 - ETA: 1:24 - loss: 1.100 - ETA: 1:19 - loss: 1.098 - ETA: 1:15 - loss: 1.098 - ETA: 1:10 - loss: 1.096 - ETA: 1:05 - loss: 1.095 - ETA: 1:01 - loss: 1.094 - ETA: 56s - loss: 1.093 - ETA: 51s - loss: 1.09 - ETA: 47s - loss: 1.09 - ETA: 42s - loss: 1.08 - ETA: 38s - loss: 1.08 - ETA: 33s - loss: 1.08 - ETA: 29s - loss: 1.08 - ETA: 24s - loss: 1.08 - ETA: 20s - loss: 1.08 - ETA: 15s - loss: 1.07 - ETA: 10s - loss: 1.07 - ETA: 6s - loss: 1.0749 - ETA: 1s - loss: 1.074 - 162s 9ms/step - loss: 1.0742 - v

17621/17621 [==============================] - ETA: 2:37 - loss: 0.738 - ETA: 2:32 - loss: 0.708 - ETA: 2:27 - loss: 0.710 - ETA: 2:23 - loss: 0.713 - ETA: 2:19 - loss: 0.725 - ETA: 2:15 - loss: 0.726 - ETA: 2:11 - loss: 0.731 - ETA: 2:06 - loss: 0.730 - ETA: 2:01 - loss: 0.728 - ETA: 1:56 - loss: 0.722 - ETA: 1:51 - loss: 0.721 - ETA: 1:46 - loss: 0.723 - ETA: 1:42 - loss: 0.724 - ETA: 1:37 - loss: 0.724 - ETA: 1:32 - loss: 0.724 - ETA: 1:27 - loss: 0.723 - ETA: 1:23 - loss: 0.723 - ETA: 1:18 - loss: 0.722 - ETA: 1:13 - loss: 0.723 - ETA: 1:08 - loss: 0.723 - ETA: 1:03 - loss: 0.721 - ETA: 59s - loss: 0.722 - ETA: 54s - loss: 0.72 - ETA: 49s - loss: 0.72 - ETA: 45s - loss: 0.72 - ETA: 40s - loss: 0.72 - ETA: 35s - loss: 0.72 - ETA: 30s - loss: 0.72 - ETA: 25s - loss: 0.72 - ETA: 21s - loss: 0.71 - ETA: 16s - loss: 0.71 - ETA: 11s - loss: 0.71 - ETA: 6s - loss: 0.7167 - ETA: 1s - loss: 0.717 - 172s 10ms/step - loss: 0.7169 - val_loss: 0.6365
Epoch 10/100
17621/17621 [==================

17621/17621 [==============================] - ETA: 2:35 - loss: 0.559 - ETA: 2:31 - loss: 0.565 - ETA: 2:26 - loss: 0.553 - ETA: 2:21 - loss: 0.567 - ETA: 2:17 - loss: 0.581 - ETA: 2:12 - loss: 0.588 - ETA: 2:08 - loss: 0.594 - ETA: 2:03 - loss: 0.591 - ETA: 1:58 - loss: 0.588 - ETA: 1:54 - loss: 0.592 - ETA: 1:49 - loss: 0.587 - ETA: 1:45 - loss: 0.585 - ETA: 1:40 - loss: 0.581 - ETA: 1:35 - loss: 0.581 - ETA: 1:30 - loss: 0.578 - ETA: 1:26 - loss: 0.578 - ETA: 1:21 - loss: 0.578 - ETA: 1:16 - loss: 0.579 - ETA: 1:12 - loss: 0.573 - ETA: 1:07 - loss: 0.575 - ETA: 1:02 - loss: 0.576 - ETA: 58s - loss: 0.576 - ETA: 53s - loss: 0.57 - ETA: 48s - loss: 0.57 - ETA: 44s - loss: 0.57 - ETA: 39s - loss: 0.57 - ETA: 34s - loss: 0.57 - ETA: 30s - loss: 0.57 - ETA: 25s - loss: 0.57 - ETA: 20s - loss: 0.58 - ETA: 15s - loss: 0.57 - ETA: 11s - loss: 0.57 - ETA: 6s - loss: 0.5786 - ETA: 1s - loss: 0.577 - 167s 9ms/step - loss: 0.5771 - val_loss: 0.5670
Epoch 18/100
17621/17621 [===================

17621/17621 [==============================] - ETA: 2:36 - loss: 0.444 - ETA: 2:31 - loss: 0.463 - ETA: 2:26 - loss: 0.463 - ETA: 2:22 - loss: 0.452 - ETA: 2:20 - loss: 0.451 - ETA: 2:16 - loss: 0.445 - ETA: 2:11 - loss: 0.450 - ETA: 2:08 - loss: 0.459 - ETA: 2:03 - loss: 0.460 - ETA: 1:58 - loss: 0.459 - ETA: 1:53 - loss: 0.457 - ETA: 1:48 - loss: 0.461 - ETA: 1:44 - loss: 0.456 - ETA: 1:39 - loss: 0.452 - ETA: 1:34 - loss: 0.452 - ETA: 1:30 - loss: 0.455 - ETA: 1:25 - loss: 0.455 - ETA: 1:20 - loss: 0.455 - ETA: 1:15 - loss: 0.460 - ETA: 1:10 - loss: 0.461 - ETA: 1:05 - loss: 0.461 - ETA: 1:00 - loss: 0.462 - ETA: 55s - loss: 0.463 - ETA: 51s - loss: 0.46 - ETA: 46s - loss: 0.46 - ETA: 41s - loss: 0.46 - ETA: 36s - loss: 0.46 - ETA: 31s - loss: 0.46 - ETA: 26s - loss: 0.46 - ETA: 21s - loss: 0.46 - ETA: 16s - loss: 0.46 - ETA: 11s - loss: 0.46 - ETA: 6s - loss: 0.4621 - ETA: 2s - loss: 0.462 - 176s 10ms/step - loss: 0.4622 - val_loss: 0.5219
Epoch 26/100
17621/17621 [================

One question could be: why do i use so much dropout? Well, fit the model with no or little dropout and you will that it starts to overfit :)

Let's see if Bi-directional LSTM can give us better results. Its a piece of cake to do it with Keras :)

In [56]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - ETA: 8:57 - loss: 1.141 - ETA: 7:01 - loss: 1.118 - ETA: 6:08 - loss: 1.117 - ETA: 5:37 - loss: 1.117 - ETA: 5:16 - loss: 1.122 - ETA: 4:58 - loss: 1.119 - ETA: 4:42 - loss: 1.115 - ETA: 4:28 - loss: 1.112 - ETA: 4:16 - loss: 1.110 - ETA: 4:05 - loss: 1.110 - ETA: 3:54 - loss: 1.108 - ETA: 3:43 - loss: 1.106 - ETA: 3:32 - loss: 1.105 - ETA: 3:21 - loss: 1.102 - ETA: 3:10 - loss: 1.100 - ETA: 3:00 - loss: 1.097 - ETA: 2:50 - loss: 1.095 - ETA: 2:40 - loss: 1.094 - ETA: 2:30 - loss: 1.091 - ETA: 2:20 - loss: 1.090 - ETA: 2:10 - loss: 1.089 - ETA: 2:00 - loss: 1.088 - ETA: 1:50 - loss: 1.085 - ETA: 1:40 - loss: 1.084 - ETA: 1:31 - loss: 1.081 - ETA: 1:21 - loss: 1.080 - ETA: 1:11 - loss: 1.078 - ETA: 1:01 - loss: 1.076 - ETA: 52s - loss: 1.073 - ETA: 42s - loss: 1.07 - ETA: 32s - loss: 1.06 - ETA: 23s - loss: 1.06 - ETA: 13s - loss: 1.06 - ETA: 4s - loss: 1.0612 - 346s 20ms/step - l

17621/17621 [==============================] - ETA: 5:38 - loss: 0.759 - ETA: 5:29 - loss: 0.718 - ETA: 5:20 - loss: 0.734 - ETA: 5:09 - loss: 0.738 - ETA: 4:59 - loss: 0.747 - ETA: 4:48 - loss: 0.739 - ETA: 4:38 - loss: 0.733 - ETA: 4:28 - loss: 0.738 - ETA: 4:17 - loss: 0.740 - ETA: 4:07 - loss: 0.735 - ETA: 3:57 - loss: 0.735 - ETA: 3:48 - loss: 0.735 - ETA: 3:38 - loss: 0.730 - ETA: 3:28 - loss: 0.728 - ETA: 3:17 - loss: 0.728 - ETA: 3:07 - loss: 0.726 - ETA: 2:57 - loss: 0.725 - ETA: 2:47 - loss: 0.725 - ETA: 2:37 - loss: 0.725 - ETA: 2:26 - loss: 0.725 - ETA: 2:16 - loss: 0.725 - ETA: 2:06 - loss: 0.726 - ETA: 1:56 - loss: 0.726 - ETA: 1:45 - loss: 0.722 - ETA: 1:35 - loss: 0.723 - ETA: 1:25 - loss: 0.724 - ETA: 1:15 - loss: 0.725 - ETA: 1:05 - loss: 0.725 - ETA: 55s - loss: 0.723 - ETA: 44s - loss: 0.72 - ETA: 34s - loss: 0.72 - ETA: 24s - loss: 0.72 - ETA: 14s - loss: 0.72 - ETA: 4s - loss: 0.7232 - 363s 21ms/step - loss: 0.7229 - val_loss: 0.6482
Epoch 10/100
17621/17621 [====

17621/17621 [==============================] - ETA: 6:35 - loss: 0.603 - ETA: 6:00 - loss: 0.591 - ETA: 5:38 - loss: 0.578 - ETA: 5:22 - loss: 0.569 - ETA: 5:10 - loss: 0.572 - ETA: 4:57 - loss: 0.571 - ETA: 4:45 - loss: 0.566 - ETA: 4:34 - loss: 0.562 - ETA: 4:23 - loss: 0.557 - ETA: 4:12 - loss: 0.561 - ETA: 4:01 - loss: 0.562 - ETA: 3:50 - loss: 0.560 - ETA: 3:40 - loss: 0.564 - ETA: 3:29 - loss: 0.567 - ETA: 3:19 - loss: 0.566 - ETA: 3:09 - loss: 0.571 - ETA: 2:58 - loss: 0.572 - ETA: 2:48 - loss: 0.572 - ETA: 2:38 - loss: 0.573 - ETA: 2:27 - loss: 0.575 - ETA: 2:17 - loss: 0.578 - ETA: 2:07 - loss: 0.582 - ETA: 1:56 - loss: 0.582 - ETA: 1:46 - loss: 0.580 - ETA: 1:36 - loss: 0.581 - ETA: 1:26 - loss: 0.582 - ETA: 1:15 - loss: 0.583 - ETA: 1:05 - loss: 0.583 - ETA: 55s - loss: 0.583 - ETA: 45s - loss: 0.58 - ETA: 34s - loss: 0.58 - ETA: 24s - loss: 0.58 - ETA: 14s - loss: 0.58 - ETA: 4s - loss: 0.5808 - 365s 21ms/step - loss: 0.5807 - val_loss: 0.5571
Epoch 18/100
17621/17621 [====

17621/17621 [==============================] - ETA: 5:38 - loss: 0.456 - ETA: 5:28 - loss: 0.440 - ETA: 5:17 - loss: 0.453 - ETA: 5:07 - loss: 0.453 - ETA: 4:57 - loss: 0.449 - ETA: 4:47 - loss: 0.453 - ETA: 4:37 - loss: 0.447 - ETA: 4:27 - loss: 0.446 - ETA: 4:17 - loss: 0.443 - ETA: 4:07 - loss: 0.444 - ETA: 3:57 - loss: 0.446 - ETA: 3:47 - loss: 0.451 - ETA: 3:37 - loss: 0.453 - ETA: 3:27 - loss: 0.452 - ETA: 3:17 - loss: 0.448 - ETA: 3:07 - loss: 0.446 - ETA: 2:56 - loss: 0.446 - ETA: 2:46 - loss: 0.444 - ETA: 2:36 - loss: 0.444 - ETA: 2:26 - loss: 0.443 - ETA: 2:16 - loss: 0.441 - ETA: 2:06 - loss: 0.441 - ETA: 1:55 - loss: 0.441 - ETA: 1:45 - loss: 0.444 - ETA: 1:35 - loss: 0.444 - ETA: 1:25 - loss: 0.443 - ETA: 1:15 - loss: 0.443 - ETA: 1:05 - loss: 0.442 - ETA: 55s - loss: 0.441 - ETA: 45s - loss: 0.44 - ETA: 34s - loss: 0.44 - ETA: 24s - loss: 0.43 - ETA: 14s - loss: 0.44 - ETA: 4s - loss: 0.4419 - 366s 21ms/step - loss: 0.4422 - val_loss: 0.5173


Pretty close! Lets try two layers of GRU:

In [57]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Train on 17621 samples, validate on 1958 samples
Epoch 1/100
17621/17621 [==============================] - ETA: 5:28 - loss: 1.128 - ETA: 4:44 - loss: 1.120 - ETA: 4:25 - loss: 1.128 - ETA: 4:11 - loss: 1.125 - ETA: 3:59 - loss: 1.129 - ETA: 3:49 - loss: 1.120 - ETA: 3:40 - loss: 1.123 - ETA: 3:31 - loss: 1.123 - ETA: 3:22 - loss: 1.123 - ETA: 3:13 - loss: 1.123 - ETA: 3:05 - loss: 1.124 - ETA: 2:57 - loss: 1.123 - ETA: 2:48 - loss: 1.122 - ETA: 2:40 - loss: 1.122 - ETA: 2:32 - loss: 1.118 - ETA: 2:24 - loss: 1.118 - ETA: 2:16 - loss: 1.116 - ETA: 2:09 - loss: 1.115 - ETA: 2:01 - loss: 1.114 - ETA: 1:53 - loss: 1.112 - ETA: 1:45 - loss: 1.110 - ETA: 1:37 - loss: 1.110 - ETA: 1:29 - loss: 1.108 - ETA: 1:21 - loss: 1.106 - ETA: 1:13 - loss: 1.104 - ETA: 1:05 - loss: 1.101 - ETA: 57s - loss: 1.101 - ETA: 50s - loss: 1.10 - ETA: 42s - loss: 1.09 - ETA: 34s - loss: 1.09 - ETA: 26s - loss: 1.09 - ETA: 18s - loss: 1.09 - ETA: 11s - loss: 1.09 - ETA: 3s - loss: 1.0893 - 279s 16ms/step - loss:

17621/17621 [==============================] - ETA: 4:31 - loss: 0.754 - ETA: 4:21 - loss: 0.748 - ETA: 4:11 - loss: 0.745 - ETA: 4:03 - loss: 0.743 - ETA: 3:56 - loss: 0.739 - ETA: 3:48 - loss: 0.744 - ETA: 3:42 - loss: 0.756 - ETA: 3:33 - loss: 0.758 - ETA: 3:25 - loss: 0.753 - ETA: 3:17 - loss: 0.750 - ETA: 3:09 - loss: 0.748 - ETA: 3:02 - loss: 0.749 - ETA: 2:54 - loss: 0.747 - ETA: 2:47 - loss: 0.747 - ETA: 2:39 - loss: 0.745 - ETA: 2:30 - loss: 0.743 - ETA: 2:22 - loss: 0.740 - ETA: 2:13 - loss: 0.742 - ETA: 2:05 - loss: 0.742 - ETA: 1:57 - loss: 0.740 - ETA: 1:49 - loss: 0.740 - ETA: 1:40 - loss: 0.740 - ETA: 1:32 - loss: 0.739 - ETA: 1:24 - loss: 0.739 - ETA: 1:16 - loss: 0.741 - ETA: 1:08 - loss: 0.739 - ETA: 1:00 - loss: 0.737 - ETA: 51s - loss: 0.738 - ETA: 43s - loss: 0.73 - ETA: 35s - loss: 0.73 - ETA: 27s - loss: 0.73 - ETA: 19s - loss: 0.73 - ETA: 11s - loss: 0.73 - ETA: 3s - loss: 0.7369 - 291s 17ms/step - loss: 0.7355 - val_loss: 0.6712
Epoch 10/100
17621/17621 [======

17621/17621 [==============================] - ETA: 4:23 - loss: 0.562 - ETA: 4:12 - loss: 0.616 - ETA: 4:05 - loss: 0.602 - ETA: 3:56 - loss: 0.599 - ETA: 3:48 - loss: 0.595 - ETA: 3:39 - loss: 0.587 - ETA: 3:31 - loss: 0.593 - ETA: 3:24 - loss: 0.593 - ETA: 3:17 - loss: 0.586 - ETA: 3:10 - loss: 0.588 - ETA: 3:02 - loss: 0.589 - ETA: 2:54 - loss: 0.589 - ETA: 2:46 - loss: 0.589 - ETA: 2:38 - loss: 0.591 - ETA: 2:30 - loss: 0.589 - ETA: 2:22 - loss: 0.592 - ETA: 2:14 - loss: 0.595 - ETA: 2:07 - loss: 0.595 - ETA: 1:59 - loss: 0.593 - ETA: 1:51 - loss: 0.591 - ETA: 1:43 - loss: 0.592 - ETA: 1:36 - loss: 0.593 - ETA: 1:28 - loss: 0.593 - ETA: 1:20 - loss: 0.596 - ETA: 1:12 - loss: 0.597 - ETA: 1:05 - loss: 0.595 - ETA: 57s - loss: 0.595 - ETA: 49s - loss: 0.59 - ETA: 41s - loss: 0.59 - ETA: 34s - loss: 0.59 - ETA: 26s - loss: 0.59 - ETA: 18s - loss: 0.59 - ETA: 10s - loss: 0.59 - ETA: 3s - loss: 0.5985 - 279s 16ms/step - loss: 0.5982 - val_loss: 0.5666
Epoch 18/100
17621/17621 [========

17621/17621 [==============================] - ETA: 4:15 - loss: 0.568 - ETA: 4:07 - loss: 0.539 - ETA: 4:01 - loss: 0.509 - ETA: 3:53 - loss: 0.501 - ETA: 3:45 - loss: 0.493 - ETA: 3:37 - loss: 0.504 - ETA: 3:30 - loss: 0.507 - ETA: 3:23 - loss: 0.499 - ETA: 3:15 - loss: 0.498 - ETA: 3:07 - loss: 0.494 - ETA: 3:00 - loss: 0.498 - ETA: 2:52 - loss: 0.501 - ETA: 2:44 - loss: 0.500 - ETA: 2:36 - loss: 0.499 - ETA: 2:29 - loss: 0.497 - ETA: 2:21 - loss: 0.495 - ETA: 2:13 - loss: 0.497 - ETA: 2:06 - loss: 0.496 - ETA: 1:58 - loss: 0.497 - ETA: 1:50 - loss: 0.496 - ETA: 1:42 - loss: 0.496 - ETA: 1:35 - loss: 0.496 - ETA: 1:27 - loss: 0.498 - ETA: 1:19 - loss: 0.499 - ETA: 1:12 - loss: 0.498 - ETA: 1:04 - loss: 0.495 - ETA: 56s - loss: 0.495 - ETA: 49s - loss: 0.49 - ETA: 41s - loss: 0.49 - ETA: 33s - loss: 0.49 - ETA: 26s - loss: 0.49 - ETA: 18s - loss: 0.49 - ETA: 10s - loss: 0.49 - ETA: 3s - loss: 0.4926 - 275s 16ms/step - loss: 0.4925 - val_loss: 0.5122
Epoch 26/100
17621/17621 [========

17621/17621 [==============================] - ETA: 4:19 - loss: 0.384 - ETA: 4:11 - loss: 0.383 - ETA: 4:02 - loss: 0.394 - ETA: 3:53 - loss: 0.406 - ETA: 3:45 - loss: 0.415 - ETA: 3:37 - loss: 0.409 - ETA: 3:30 - loss: 0.410 - ETA: 3:22 - loss: 0.407 - ETA: 3:14 - loss: 0.402 - ETA: 3:07 - loss: 0.403 - ETA: 2:59 - loss: 0.401 - ETA: 2:51 - loss: 0.404 - ETA: 2:44 - loss: 0.404 - ETA: 2:36 - loss: 0.403 - ETA: 2:29 - loss: 0.401 - ETA: 2:21 - loss: 0.404 - ETA: 2:13 - loss: 0.401 - ETA: 2:06 - loss: 0.402 - ETA: 1:58 - loss: 0.401 - ETA: 1:50 - loss: 0.400 - ETA: 1:42 - loss: 0.401 - ETA: 1:35 - loss: 0.401 - ETA: 1:27 - loss: 0.402 - ETA: 1:19 - loss: 0.401 - ETA: 1:12 - loss: 0.400 - ETA: 1:04 - loss: 0.401 - ETA: 56s - loss: 0.399 - ETA: 49s - loss: 0.40 - ETA: 41s - loss: 0.40 - ETA: 33s - loss: 0.39 - ETA: 26s - loss: 0.39 - ETA: 18s - loss: 0.39 - ETA: 10s - loss: 0.39 - ETA: 3s - loss: 0.3981 - 276s 16ms/step - loss: 0.3981 - val_loss: 0.5028
Epoch 34/100
17621/17621 [========

Nice! Much better than what we had previously! Keep optimizing and the performance will keep improving.
Worth trying: stemming and lemmatization. This is something I'm skipping for now.

In the Kaggle world, to get a top score you should have an ensemble of models. Let's check a little bit of ensembling!


## Ensembling

Few months back I made a simple ensembler but I didn't have time to develop it fully. It can be found here: https://github.com/abhishekkrthakur/pysembler . I'm going to use some part of it here:

In [58]:
# this is the main ensembling class. how to use it is in the next cell!
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
import os
import sys
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format="[%(asctime)s] %(levelname)s %(message)s",
    datefmt="%H:%M:%S", stream=sys.stdout)
logger = logging.getLogger(__name__)


class Ensembler(object):
    def __init__(self, model_dict, num_folds=3, task_type='classification', optimize=roc_auc_score,
                 lower_is_better=False, save_path=None):
        """
        Ensembler init function
        :param model_dict: model dictionary, see README for its format
        :param num_folds: the number of folds for ensembling
        :param task_type: classification or regression
        :param optimize: the function to optimize for, e.g. AUC, logloss, etc. Must have two arguments y_test and y_pred
        :param lower_is_better: is lower value of optimization function better or higher
        :param save_path: path to which model pickles will be dumped to along with generated predictions, or None
        """

        self.model_dict = model_dict
        self.levels = len(self.model_dict)
        self.num_folds = num_folds
        self.task_type = task_type
        self.optimize = optimize
        self.lower_is_better = lower_is_better
        self.save_path = save_path

        self.training_data = None
        self.test_data = None
        self.y = None
        self.lbl_enc = None
        self.y_enc = None
        self.train_prediction_dict = None
        self.test_prediction_dict = None
        self.num_classes = None

    def fit(self, training_data, y, lentrain):
        """
        :param training_data: training data in tabular format
        :param y: binary, multi-class or regression
        :return: chain of models to be used in prediction
        """

        self.training_data = training_data
        self.y = y

        if self.task_type == 'classification':
            self.num_classes = len(np.unique(self.y))
            logger.info("Found %d classes", self.num_classes)
            self.lbl_enc = LabelEncoder()
            self.y_enc = self.lbl_enc.fit_transform(self.y)
            kf = StratifiedKFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, self.num_classes)
        else:
            self.num_classes = -1
            self.y_enc = self.y
            kf = KFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, 1)

        self.train_prediction_dict = {}
        for level in range(self.levels):
            self.train_prediction_dict[level] = np.zeros((train_prediction_shape[0],
                                                          train_prediction_shape[1] * len(self.model_dict[level])))

        for level in range(self.levels):

            if level == 0:
                temp_train = self.training_data
            else:
                temp_train = self.train_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):
                validation_scores = []
                foldnum = 1
                for train_index, valid_index in kf.split(self.train_prediction_dict[0], self.y_enc):
                    logger.info("Training Level %d Fold # %d. Model # %d", level, foldnum, model_num)

                    if level != 0:
                        l_training_data = temp_train[train_index]
                        l_validation_data = temp_train[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])
                    else:
                        l0_training_data = temp_train[0][model_num]
                        if type(l0_training_data) == list:
                            l_training_data = [x[train_index] for x in l0_training_data]
                            l_validation_data = [x[valid_index] for x in l0_training_data]
                        else:
                            l_training_data = l0_training_data[train_index]
                            l_validation_data = l0_training_data[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])

                    logger.info("Predicting Level %d. Fold # %d. Model # %d", level, foldnum, model_num)

                    if self.task_type == 'classification':
                        temp_train_predictions = model.predict_proba(l_validation_data)
                        self.train_prediction_dict[level][valid_index,
                        (model_num * self.num_classes):(model_num * self.num_classes) +
                                                       self.num_classes] = temp_train_predictions

                    else:
                        temp_train_predictions = model.predict(l_validation_data)
                        self.train_prediction_dict[level][valid_index, model_num] = temp_train_predictions
                    validation_score = self.optimize(self.y_enc[valid_index], temp_train_predictions)
                    validation_scores.append(validation_score)
                    logger.info("Level %d. Fold # %d. Model # %d. Validation Score = %f", level, foldnum, model_num,
                                validation_score)
                    foldnum += 1
                avg_score = np.mean(validation_scores)
                std_score = np.std(validation_scores)
                logger.info("Level %d. Model # %d. Mean Score = %f. Std Dev = %f", level, model_num,
                            avg_score, std_score)

            logger.info("Saving predictions for level # %d", level)
            train_predictions_df = pd.DataFrame(self.train_prediction_dict[level])
            train_predictions_df.to_csv(os.path.join(self.save_path, "train_predictions_level_" + str(level) + ".csv"),
                                        index=False, header=None)

        return self.train_prediction_dict

    def predict(self, test_data, lentest):
        self.test_data = test_data
        if self.task_type == 'classification':
            test_prediction_shape = (lentest, self.num_classes)
        else:
            test_prediction_shape = (lentest, 1)

        self.test_prediction_dict = {}
        for level in range(self.levels):
            self.test_prediction_dict[level] = np.zeros((test_prediction_shape[0],
                                                         test_prediction_shape[1] * len(self.model_dict[level])))
        self.test_data = test_data
        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
                temp_test = self.test_data
            else:
                temp_train = self.train_prediction_dict[level - 1]
                temp_test = self.test_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):

                logger.info("Training Fulldata Level %d. Model # %d", level, model_num)
                if level == 0:
                    model.fit(temp_train[0][model_num], self.y_enc)
                else:
                    model.fit(temp_train, self.y_enc)

                logger.info("Predicting Test Level %d. Model # %d", level, model_num)

                if self.task_type == 'classification':
                    if level == 0:
                        temp_test_predictions = model.predict_proba(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict_proba(temp_test)
                    self.test_prediction_dict[level][:, (model_num * self.num_classes): (model_num * self.num_classes) +
                                                                                        self.num_classes] = temp_test_predictions

                else:
                    if level == 0:
                        temp_test_predictions = model.predict(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict(temp_test)
                    self.test_prediction_dict[level][:, model_num] = temp_test_predictions

            test_predictions_df = pd.DataFrame(self.test_prediction_dict[level])
            test_predictions_df.to_csv(os.path.join(self.save_path, "test_predictions_level_" + str(level) + ".csv"),
                                       index=False, header=None)

        return self.test_prediction_dict


In [59]:
# specify the data to be used for every level of ensembling:
train_data_dict = {0: [xtrain_tfv, xtrain_ctv, xtrain_tfv, xtrain_ctv], 1: [xtrain_glove]}
test_data_dict = {0: [xvalid_tfv, xvalid_ctv, xvalid_tfv, xvalid_ctv], 1: [xvalid_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],

              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='')

ens.fit(train_data_dict, ytrain, lentrain=xtrain_glove.shape[0])
preds = ens.predict(test_data_dict, lentest=xvalid_glove.shape[0])

[19:39:59] INFO Found 3 classes
[19:39:59] INFO Training Level 0 Fold # 1. Model # 0
[19:40:00] INFO Predicting Level 0. Fold # 1. Model # 0
[19:40:00] INFO Level 0. Fold # 1. Model # 0. Validation Score = 0.679328
[19:40:00] INFO Training Level 0 Fold # 2. Model # 0
[19:40:00] INFO Predicting Level 0. Fold # 2. Model # 0
[19:40:00] INFO Level 0. Fold # 2. Model # 0. Validation Score = 0.670841
[19:40:00] INFO Training Level 0 Fold # 3. Model # 0
[19:40:00] INFO Predicting Level 0. Fold # 3. Model # 0
[19:40:00] INFO Level 0. Fold # 3. Model # 0. Validation Score = 0.672830
[19:40:00] INFO Level 0. Model # 0. Mean Score = 0.674333. Std Dev = 0.003624
[19:40:00] INFO Training Level 0 Fold # 1. Model # 1
[19:40:03] INFO Predicting Level 0. Fold # 1. Model # 1
[19:40:03] INFO Level 0. Fold # 1. Model # 1. Validation Score = 0.574757
[19:40:03] INFO Training Level 0 Fold # 2. Model # 1
[19:40:05] INFO Predicting Level 0. Fold # 2. Model # 1
[19:40:05] INFO Level 0. Fold # 2. Model # 1. Val

In [60]:
# check error:
multiclass_logloss(yvalid, preds[1])

0.42240600232369452

Thus, we see that ensembling improves the score by a great extent! Since this is supposed to be a tutorial only I wont be providing any CSVs that you can submit to the leaderboard.

I hope you like it! 

P.S.: If the response is good, I'll add more stuff in this! :)